In [6]:
from pyspark.ml.clustering import KMeans
from pyspark import SparkContext, SparkConf

#cretion of spark session

spark = SparkSession \
    .builder \
    .appName("Python Spark ML example") \
    .getOrCreate()
# Loads data.
dataset= spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("adult5.csv")

# Trains a k-means model.
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


#creation of the features vector and transforming the dataset for the same
workclass_stringIndexer = StringIndexer(inputCol="workclass", outputCol="workclass_indexed")
si_model_workclass = workclass_stringIndexer.fit(dataset)
education_stringIndexer = StringIndexer(inputCol="education", outputCol="education_indexed")
si_model_education = education_stringIndexer.fit(dataset)
occupation_stringIndexer = StringIndexer(inputCol="occupation", outputCol="occupation_indexed")
si_model_occupation = occupation_stringIndexer.fit(dataset)
sex_stringIndexer = StringIndexer(inputCol="sex", outputCol="sex_indexed")
si_model_sex = sex_stringIndexer.fit(dataset)
income_stringIndexer = StringIndexer(inputCol="income", outputCol="income_indexed")
si_model_income = income_stringIndexer.fit(dataset)
td = si_model_workclass.transform(si_model_education.transform(si_model_occupation.transform(si_model_sex.transform(si_model_income.transform(dataset)))))
td.toPandas()
from pyspark.ml.feature import VectorAssembler

feature_columns = ["age","workclass_indexed","fnlwgt","education_num","hours_per_week","sex_indexed","education_indexed","occupation_indexed","income_indexed"]

feature_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

td_dataset_prepared = feature_assembler.transform(td)
td_dataset_prepared.printSchema()
td_dataset_prepared.toPandas()


#making the k_means model
new1=td_dataset_prepared.select("features")
#new1.show()
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(new1)
transformed = model.transform(td_dataset_prepared)
transformed.toPandas()


#calculation of SSE
wssse = model.computeCost(td_dataset_prepared)
print("Within Set Sum of Squared Errors = " + str(wssse))


#Computing the final clusters of the dataset
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- occupation: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- income: string (nullable = true)
 |-- income_indexed: double (nullable = true)
 |-- sex_indexed: double (nullable = true)
 |-- occupation_indexed: double (nullable = true)
 |-- education_indexed: double (nullable = true)
 |-- workclass_indexed: double (nullable = true)
 |-- features: vector (nullable = true)

Within Set Sum of Squared Errors = 2964380903.0
Cluster Centers: 
[  3.10000000e+01   3.33333333e-01   8.18563333e+04   1.33333333e+01
   4.00000000e+01   6.66666667e-01   3.33333333e-01   1.00000000e+00
   3.33333333e-01]
[  4.20000000e+01   1.00000000e+00   2.07330500e+05   1.05000000e+01
   4.75000000e+01   0.00000000e+00   2.50000000e+00   1.5000